In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import sys
import pandas as pd


import random

In [ ]:
dataPred = pd.read_csv('../Prediction_Files/predictions_KNN.csv')

In [65]:
minVals = dataPred.min()
maxVals = dataPred.max()

max(dataPred.Cost - minVals.Cost/ maxVals.Cost - minVals.Cost)

0.7019305019971172

# reading predictions and transforming data

In [134]:
def rowToTime(row):
    timeAtMoment = row.Hours*3600*1000
    timeAtMoment += row.Minutes*60*1000
    timeAtMoment += row.Seconds*1000
    return timeAtMoment

def shiftTransformCol50(data, transformation = 50):
    return (data + abs(data.min()))*50

def utilFunction(latency, cost, reward = 100, reliability = 1):
    epsilon = 0.00000000000000000000000000000000000000000000001

    if reliability == 0:
        return 0
    else:
        return ((reward)/( (2*latency) + cost + epsilon )) * reliability 
    

def celia(data,updateThreshold = 1000):
    lastUpdate = 0;
    utilitPredicted = []
    utilityReal = []
    award = []
    lostUtility = []
    shouldHaveUpdated = []
    performedUpdate = []

    temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']
    numOfUpdates = 0
    negativeAward = []
    for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
        timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[lastUpdate]['Timestamp'])

        predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predCost,(float)(timeDeltaReward), data.iloc[i].predReliability )

        realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
        
        utilitPredicted.append(predictedUtil)
        
        utilityReal.append(realUtility)
        
        
        if predictedUtil >= updateThreshold:
    #         award.append(award[i-1] + realUtility-1);
#             award.append(realUtility-updateThreshold);

            lastUpdate = i;
            lostUtility.append(0)
            
            performedUpdate.append(1) #this is the should/not have updated checks 
            
            if realUtility >= updateThreshold:
                award.append(realUtility-updateThreshold);
                negativeAward.append(0)
                shouldHaveUpdated.append(1)
            else:
                award.append(0)
                shouldHaveUpdated.append(0)
                negativeAward.append(abs(realUtility-updateThreshold));
                
            numOfUpdates += 1
        else:
    #         award.append(award[i-1]);
            award.append(0);
            
            performedUpdate.append(0)
            
            if realUtility >= updateThreshold:
                lostUtility.append(realUtility)
                negativeAward.append(realUtility)

                shouldHaveUpdated.append(1)

            else:
                lostUtility.append(0) 
                
                negativeAward.append(0)

                shouldHaveUpdated.append(0)

    
    results = pd.DataFrame(np.stack([utilitPredicted,utilityReal, award, negativeAward, lostUtility,shouldHaveUpdated,performedUpdate],axis=1),columns=['predictedUtility','realUtility','award','negativeAward','lostUtility','shouldHaveUpdated','performedUpdate'])
    return results, numOfUpdates
#     results.to_csv("./results/dummyRandom.csv")

def celiaSim(dataPred, updateThreshold = 1000):
    ### this is converting hours, minutes, seconds to a millisecond time series
    dataPred.Hours = ((dataPred.Hours)*23)+0
    dataPred.Minutes = ((dataPred.Minutes)*59)+0
    dataPred.Seconds = ((dataPred.Seconds)*59.999805)+0.1025061111111
    
    cumulativeTime = [rowToTime(dataPred.iloc[0])]
    resetAddTime = 23*3600*1000
    resetAddTime += 59*60*1000
    resetAddTime += 60*1000
        
    for i in range(1,len(dataPred)):
        timeAtMoment = rowToTime(dataPred.iloc[i])
        lastTimeMoment = rowToTime(dataPred.iloc[i-1])
        if timeAtMoment > lastTimeMoment:
            cumuTime = cumulativeTime[i-1] + (timeAtMoment - lastTimeMoment)
            cumulativeTime.append(cumuTime)
        else:
            cumulativeTime.append(cumulativeTime[i-1]+(timeAtMoment+resetAddTime)-lastTimeMoment)
    #         print(cumulativeTime[i])
    
    cumulativeTime = cumulativeTime- cumulativeTime[0]
    
    ### shifting and transforming data to a range of 0-50
    newData = np.stack((cumulativeTime, shiftTransformCol50(dataPred.Latency), shiftTransformCol50(dataPred.Total_Cost), dataPred.Reliability, shiftTransformCol50(dataPred.predicted_Latency), shiftTransformCol50(dataPred.predicted_Total_Cost), round(dataPred.predicted_Reliability)),axis = 1)
    newData = pd.DataFrame(newData, columns=['Timestamp', 'Latency', 'Cost', 'Reliability', 'predLatency', 'predCost', 'predReliability' ])

    return celia(newData, updateThreshold)
#     return newData

In [ ]:
result = celiaSim(dataPred)

In [90]:
len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 1])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


97

In [136]:
# pd.read_csv('../CELIA/dummy/dummy30'+'%.csv')

In [151]:
muls = [1,0.9,0.7,0.5,1.1,1.3,1.5,1.8,2.1]
names = [0,-10,-30,-50,10,30,50,80,110]
# muls = [1]
# names = [0]
stats = []
for i in range(len(muls)):
    data = pd.read_csv('../CELIA/dummy/dummy'+str(names[i])+'%.csv')
    result, numOfUpdates = celia(data, 154)
#     result.to_csv(outputDirectory+'celiaResult_'+i)
    utilDif = np.sum(abs(result.predictedUtility-result.realUtility))
    award = sum(result.award)
    negativeAward = sum(result.negativeAward)
    lostUtil = sum(result.lostUtility)
    
    decisionShouldHaveDid = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 1])
    decisionShouldHaveDidNot = len(result[result.shouldHaveUpdated == 1][result.performedUpdate == 0])
    decisionNotHaveDid = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 1])
    decisionNotHaveDidNot = len(result[result.shouldHaveUpdated == 0][result.performedUpdate == 0])
    
    wrongDecisions = decisionShouldHaveDidNot + decisionNotHaveDid
    correctDecisions = decisionShouldHaveDid + decisionNotHaveDidNot
    
    stats.append([names[i], utilDif, award, negativeAward, numOfUpdates, lostUtil, wrongDecisions, correctDecisions, decisionShouldHaveDid, decisionShouldHaveDidNot, decisionNotHaveDid, decisionNotHaveDidNot])
    


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [152]:
stats = pd.DataFrame(stats, columns=['predictionType','utilDif','reward','negativeReward', 'numOfUpdates','lostUtil', 'wrongDecisions', 'correctDecisions', 'decisionShouldHaveDid', 'decisionShouldHaveDidNot', 'decisionNotHaveDid', 'decisionNotHaveDidNot'])

In [153]:
stats.to_csv('./dummy/dummyStats.csv')

In [154]:
stats

,predictionType,utilDif,reward,negativeReward,numOfUpdates,lostUtil,wrongDecisions,correctDecisions,decisionShouldHaveDid,decisionShouldHaveDidNot,decisionNotHaveDid,decisionNotHaveDidNot
0,0,0.000000e+00,1.067045e+09,0.000000,5289,0.000000,0,11488,5289,0,0,6199
1,-10,1.187053e+08,1.067011e+09,2916.422805,5593,0.000000,339,11149,5254,0,339,5895
2,-30,4.577132e+08,1.066863e+09,68856.699969,7115,0.000000,2012,9476,5103,0,2012,4373
3,-50,1.067935e+09,1.066841e+09,86973.655337,7361,0.000000,2261,9227,5100,0,2261,4127
4,10,9.712855e+07,1.067067e+09,30247.224152,5109,30247.224152,187,11301,5109,187,0,6192
5,30,2.465598e+08,1.067077e+09,44203.816459,5031,44203.816459,265,11223,5031,265,0,6192
6,50,3.561439e+08,1.067081e+09,50857.282447,5000,50857.282447,296,11192,5000,296,0,6192
7,80,4.748610e+08,1.067084e+09,57518.472317,4974,57518.472317,322,11166,4974,322,0,6192
8,110,5.596619e+08,1.067089e+09,65841.540897,4950,65841.540897,350,11138,4950,350,0,6188


realUtilityMean = 4.138921e+06	
realUtilStd = 6.175693e+06	

In [ ]:
lastUpdate = 0;
updateThreshold = 1000
utilitPredicted = []
utilityReal = []
award = []
lostUtility = []

In [ ]:
# real time utility aware analysis tool

In [ ]:
temp = data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp']
# type(temp.total_seconds())
# pd.to_datetime(data.iloc[1]['Timestamp'] - data.iloc[0]['Timestamp'], unit='ms')

In [ ]:
for i in range(1,len(data)):
#     timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[i-1]['Timestamp']).total_seconds()
    timeDeltaReward = (data.iloc[i]['Timestamp'] - data.iloc[lastUpdate]['Timestamp'])

    predictedUtil = utilFunction(data.iloc[i].predLatency, data.iloc[i].predCost,(float)(timeDeltaReward), data.iloc[i].predReliability )
    
    realUtility = utilFunction(data.iloc[i].Latency, data.iloc[i].Cost,(float)(timeDeltaReward), data.iloc[i].Reliability )
    utilitPredicted.append(predictedUtil)
    utilityReal.append(realUtility)
    if predictedUtil >= updateThreshold:
#         award.append(award[i-1] + realUtility-1);
        award.append(realUtility-updateThreshold);
        lastUpdate = i;
        lostUtility.append(0)
    else:
#         award.append(award[i-1]);
        award.append(0);
    
        if realUtility >= updateThreshold:
            lostUtility.append(realUtility)
        else:
            lostUtility.append(0)


In [ ]:
results = pd.DataFrame(np.stack([utilitPredicted,utilityReal,award,lostUtility],axis=1),columns=['predictedUtility','realUtility','award','lostUtility'])

In [ ]:
results.describe()

In [ ]:
results

In [ ]:
results.to_csv("./results/dummyRandom.csv")

In [ ]:
cumulativeTime[20]-cumulativeTime[19]